In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2020-11-03 09:52:51--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.253.63.128, 142.250.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.001s  

2020-11-03 09:52:51 (124 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



In [4]:
data = open('/tmp/irish-lyrics-eof.txt').read()

In [7]:
data[:100]

'Come all ye maidens young and fair\nAnd you that are blooming in your prime\nAlways beware and keep yo'

In [8]:
corpus = data.lower().split("\n")

In [10]:
corpus[0:10]

['come all ye maidens young and fair',
 'and you that are blooming in your prime',
 'always beware and keep your garden fair',
 'let no man steal away your thyme',
 'for thyme it is a precious thing',
 'and thyme brings all things to my mind',
 'nlyme with all its flavours, along with all its joys',
 'thyme, brings all things to my mind',
 'once i and a bunch of thyme',
 'i thought it never would decay']

In [11]:
tok = Tokenizer()

In [12]:
tok.fit_on_texts(corpus)

In [13]:
total_words = len(tok.word_index) + 1 #1 for out of vocabulary 

In [19]:
total_words

2690

In [22]:
input_sequences = []
for line in corpus:
	token_list = tok.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

In [23]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [28]:
input_sequences[0,:]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 51, 12],
      dtype=int32)

In [32]:
print(input_sequences[0, :-1],'\t\t', input_sequences[0, -1])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 51] 		 12


In [33]:
# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [41]:
labels[0],ys[0, 0:13]

(12,
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32))

In [43]:
model = Sequential()
model.add(Embedding(total_words, 240, input_length=max_sequence_len-1)) #max_sequence_len-1 because the last sequence code is taken as label i.e. input_sequences[:,-1]
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))

adam = Adam(lr=0.01)

In [44]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
#print model.summary()
print(model)

Epoch 1/100
377/377 [==============================] - 4s 12ms/step - loss: 6.6386 - accuracy: 0.0763
Epoch 2/100
377/377 [==============================] - 4s 11ms/step - loss: 5.8597 - accuracy: 0.1141
Epoch 3/100
377/377 [==============================] - 4s 11ms/step - loss: 6.3924 - accuracy: 0.1282
Epoch 4/100
377/377 [==============================] - 4s 12ms/step - loss: 4.7030 - accuracy: 0.1853
Epoch 5/100
377/377 [==============================] - 4s 12ms/step - loss: 3.8659 - accuracy: 0.2575
Epoch 6/100
377/377 [==============================] - 4s 12ms/step - loss: 3.2743 - accuracy: 0.3295
Epoch 7/100
377/377 [==============================] - 4s 11ms/step - loss: 2.8342 - accuracy: 0.3917
Epoch 8/100
377/377 [==============================] - 4s 12ms/step - loss: 2.5457 - accuracy: 0.4366
Epoch 9/100
377/377 [==============================] - 4s 11ms/step - loss: 2.2293 - accuracy: 0.4951
Epoch 10/100
377/377 [==============================] - 4s 11ms/step - loss: 2.056

In [67]:
seed_text = "I see a beautiful river"
next_words = 50
  
for _ in range(next_words):
    token_list = tok.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted, axis =1)


    output_word = ""
    for word, index in tok.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text = seed_text + " " + output_word

print(seed_text)

I see a beautiful river here in the land of mooncoin dead and laid out on comes laughing down break far away from spancil hill says along with thee gone years says sure have none deceive me or near but rest with rover from auld rigadoo hearty dying hearty dying dying shannon grow winds love


In [ ]:
import jovian
jovian.commit(project='nlp-poem-generation')

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
